In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import stan
from itertools import combinations

In [2]:
df = pd.read_csv("./curso-55N-2024.csv", delimiter=";", skipinitialspace=True)

df = df[df["ANO_EVASAO"].str.match(r'\d+$')]
df = df[df["ANO_INGRESSO"].str.match(r'\d+$')]

In [4]:
df = df.apply(lambda x : pd.factorize(x)[0])

corr = df.corr(method='pearson', min_periods=1)

corr[["NUM_PERIODO_ALUNO", "FORMA_EVASAO"]].dropna()

,NUM_PERIODO_ALUNO,FORMA_EVASAO
COTISTA,-0.067028,0.044230
TIPO_COTA,-0.002110,0.000063
MATR_ALUNO,0.073562,0.041621
ANO_EVASAO,-0.006223,0.076353
PERIODO_EVASAO,-0.032738,0.040234
FORMA_EVASAO,-0.082201,1.000000
ANO_INGRESSO,0.065040,0.001992
DT_SAIDA,0.024397,0.034896
SEXO,0.010655,-0.087184
PERIODO_INGRESSO,0.054325,0.161853


In [5]:
correlation_model = """
data {
  int<lower=0> N;  
  vector[N] X;     
  vector[N] Y;     
}

parameters {
  real beta0;     
  real beta1;     
  real<lower=0> sigma; 
}

model {
  beta0 ~ normal(0, 10);
  beta1 ~ normal(0, 10); 
  sigma ~ cauchy(0, 5);

  Y ~ normal(beta0 + beta1 * X, sigma);
}

generated quantities {
  real beta1_rep;  
  beta1_rep = normal_rng(beta1, 0.1);
}
"""

In [6]:
semesters_beta1 = []
dropout_beta1 = []
columns = []

for column, series in df.items():

    series = series.to_numpy()
    semesters = df["NUM_PERIODO_ALUNO"].to_numpy()
    dropout = df["FORMA_EVASAO"].to_numpy()

    reg1_data = {
        'N': len(series),
        'X': series,
        'Y': semesters 
    }
    
    reg2_data = {
        'N': len(series),
        'X': series,
        'Y': dropout 
    }

    posterior1 = stan.build(correlation_model, reg1_data)
    posterior2 = stan.build(correlation_model, reg2_data)

    fit1 = posterior1.sample(num_chains=4, num_samples=1000)
    fit2 = posterior2.sample(num_chains=4, num_samples=1000)

    beta1_test1 = fit1.to_frame()["beta1_rep"].mean()
    beta1_test2 = fit2.to_frame()["beta1_rep"].mean()

    columns.append(column)
    semesters_beta1.append(beta1_test1)
    dropout_beta1.append(beta1_test2)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
orr_df = pd.DataFrame({
  "COLUMNS": columns,
  "NUM_PERIODO_ALUNO": semesters_beta1,
  "FORMA_EVASAO": dropout_beta1
})

corr_df

NameError: name 'corr_df' is not defined